In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

In [2]:
ds_snow = xr.open_mfdataset("/data/shared_data/NNA/NNA.4km.hERA5.1989.003/snow_m/NNA.4km.hERA5.1989.003.clm2.h0.SNOW.****.nc", 
                            chunks={'time': 1})
ds_rain = xr.open_mfdataset("/data/shared_data/NNA/NNA.4km.hERA5.1989.003/rain_m/NNA.4km.hERA5.1989.003.clm2.h0.RAIN.****.nc")
ds_qrunoff = xr.open_mfdataset("/data/shared_data/NNA/NNA.4km.hERA5.1989.003/qrunoff_m/NNA.4km.hERA5.1989.003.clm2.h0.QRUNOFF.****.nc")

In [3]:
ds_snow.load()
ds_rain.load()
ds_qrunoff.load()

<xarray.Dataset> Size: 803MB
Dimensions:      (time: 388, hist_interval: 2, lat: 662, lon: 782)
Coordinates:
  * time         (time) datetime64[ns] 3kB 1989-07-01 1989-08-01 ... 2021-10-01
  * lon          (lon) float32 3kB 184.2 184.3 184.3 184.4 ... 225.7 225.7 225.8
  * lat          (lat) float32 3kB 50.01 50.04 50.07 50.11 ... 69.83 69.86 69.89
Dimensions without coordinates: hist_interval
Data variables:
    time_bounds  (time, hist_interval) datetime64[ns] 6kB 1989-05-31T23:50:00...
    QRUNOFF      (time, lat, lon) float32 803MB nan nan ... -1.704e-06
Attributes: (12/39)
    title:                                     CLM History file information
    comment:                                   NOTE: None of the variables ar...
    Conventions:                               CF-1.0
    history:                                   Thu Feb 13 10:30:33 2025: ncrc...
    source:                                    Community Terrestrial Systems ...
    hostname:                                  cheyenne
    ...                                        ...
    ctype_urban_impervious_road:               74
    ctype_urban_pervious_road:                 75
    cft_c3_crop:                               1
    cft_c3_irrigated:                          2
    time_period_freq:                          month_1
    NCO:                                       netCDF Operators version 5.2.4...

In [4]:
ds_climate_region = xr.open_dataset("/data/shared_data/NNA/BAMS/alaska_climate_region.nc")
ds_precip = ds_rain.RAIN + ds_snow.SNOW

In [5]:
def winterizer(ds):
    """Takes a dataset in, returns a version with means for the winter months."""
    djfm = ds.sel(time=ds.time.dt.month.isin([12, 1, 2, 3]))

    def get_season_year(time):
        month = time.dt.month
        year = time.dt.year
        return xr.DataArray(
            year.where(month != 12, year + 1),
            dims='time'
        )
    
    djfm.coords['season_year'] = get_season_year(djfm.time).astype('int32')

    return djfm.groupby('season_year').mean(dim=('lat','lon', 'time'))

def summerizer(ds):
    """Takes a dataset in, returns a version with means for the summer months."""
    mjja = ds.sel(time=ds.time.dt.month.isin([5, 6, 7, 8]))

    def get_season_year(time):
        month = time.dt.month
        year = time.dt.year
        return xr.DataArray(
            year.where(month != 12, year + 1),
            dims='time'
        )
    mjja.coords['season_year'] = get_season_year(mjja.time).astype('int32')
    
    return mjja.groupby('season_year').mean(dim=('lat','lon', 'time'))

In [6]:
djfm_precip = winterizer(ds_precip.where((ds_climate_region.OBJECTID.values == 2)))
djfm_qrunoff = winterizer(ds_qrunoff.QRUNOFF.where(ds_climate_region.OBJECTID.values == 2)) 

In [7]:
djfm_precip

<xarray.DataArray (season_year: 32)> Size: 128B
array([6.7640176e-05, 5.9490449e-05, 5.0152576e-05, 6.7392466e-05,
       6.1854036e-05, 6.0442122e-05, 5.5192239e-05, 5.2224965e-05,
       4.9158753e-05, 6.6661749e-05, 6.6551394e-05, 7.9573125e-05,
       7.5114076e-05, 6.3017847e-05, 6.2936131e-05, 6.9616995e-05,
       5.6494559e-05, 5.6951209e-05, 7.1103110e-05, 6.3196305e-05,
       5.3177580e-05, 5.0423278e-05, 7.3014213e-05, 6.6835491e-05,
       7.1135946e-05, 5.1966919e-05, 6.9951799e-05, 5.3991494e-05,
       6.2556450e-05, 7.1800438e-05, 6.4266656e-05, 6.9138958e-05],
      dtype=float32)
Coordinates:
  * season_year  (season_year) int32 128B 1990 1991 1992 1993 ... 2019 2020 2021

In [8]:
djfm_qrunoff

<xarray.DataArray 'QRUNOFF' (season_year: 32)> Size: 128B
array([2.3147999e-05, 4.0419440e-05, 2.0157580e-05, 4.1285348e-05,
       3.4119679e-05, 3.2599120e-05, 3.2351236e-05, 2.9802473e-05,
       1.6759250e-05, 3.4557172e-05, 2.0071753e-05, 4.0761690e-05,
       3.1277741e-05, 3.7619535e-05, 2.9899638e-05, 4.1395091e-05,
       2.6093277e-05, 2.8459828e-05, 3.3162392e-05, 3.2574899e-05,
       2.9498244e-05, 3.0217199e-05, 3.0521827e-05, 2.8994655e-05,
       4.9325969e-05, 3.1375090e-05, 3.1018095e-05, 2.9088840e-05,
       3.8937877e-05, 4.5941902e-05, 3.3613502e-05, 3.8888757e-05],
      dtype=float32)
Coordinates:
  * season_year  (season_year) int32 128B 1990 1991 1992 1993 ... 2019 2020 2021
Attributes:
    long_name:     total liquid runoff not including correction for land use ...
    units:         mm/s
    cell_methods:  time: mean

In [9]:
mjja_precip = summerizer(ds_precip.sel(time=slice("1990-05-01","2021-10-01")).where((ds_climate_region.OBJECTID.values == 2)))
mjja_qrunoff = summerizer(ds_qrunoff.QRUNOFF.sel(time=slice("1990-05-01","2021-10-01")).where(ds_climate_region.OBJECTID.values == 2))

In [10]:
mjja_precip

<xarray.DataArray (season_year: 32)> Size: 128B
array([4.1294006e-05, 4.2709733e-05, 3.3754161e-05, 4.6819350e-05,
       4.3636137e-05, 3.8472961e-05, 4.7345591e-05, 2.9247549e-05,
       5.0366525e-05, 3.7494923e-05, 3.9334049e-05, 3.3664888e-05,
       4.0680105e-05, 4.4046716e-05, 4.8048983e-05, 4.5485205e-05,
       3.6747431e-05, 3.2067172e-05, 4.0369170e-05, 4.8693561e-05,
       4.3267919e-05, 5.8535894e-05, 3.7974576e-05, 4.3062078e-05,
       2.7114471e-05, 4.4355656e-05, 4.1594609e-05, 4.1431249e-05,
       4.8809216e-05, 4.0037281e-05, 3.1955951e-05, 4.6662644e-05],
      dtype=float32)
Coordinates:
  * season_year  (season_year) int32 128B 1990 1991 1992 1993 ... 2019 2020 2021

In [11]:
mjja_qrunoff

<xarray.DataArray 'QRUNOFF' (season_year: 32)> Size: 128B
array([6.1326944e-05, 4.2986241e-05, 5.7588026e-05, 5.4569598e-05,
       6.0915932e-05, 5.0021215e-05, 5.3188240e-05, 3.9893934e-05,
       6.0363138e-05, 6.2820109e-05, 7.0919654e-05, 5.8256559e-05,
       6.6040615e-05, 5.3480257e-05, 6.6316119e-05, 5.3518630e-05,
       5.4007283e-05, 4.2971504e-05, 6.2105682e-05, 6.6574539e-05,
       5.5706274e-05, 6.1577572e-05, 6.5094398e-05, 7.3187890e-05,
       4.0992087e-05, 5.1632422e-05, 6.4640226e-05, 4.8832215e-05,
       5.0652059e-05, 4.9698341e-05, 4.5792047e-05, 6.0907500e-05],
      dtype=float32)
Coordinates:
  * season_year  (season_year) int32 128B 1990 1991 1992 1993 ... 2019 2020 2021
Attributes:
    long_name:     total liquid runoff not including correction for land use ...
    units:         mm/s
    cell_methods:  time: mean

In [12]:
djfm_qrunoff/djfm_precip

<xarray.DataArray (season_year: 32)> Size: 128B
array([0.34222263, 0.6794274 , 0.40192512, 0.61261076, 0.5516161 ,
       0.53934443, 0.58615553, 0.5706557 , 0.34092098, 0.51839584,
       0.30159777, 0.5122545 , 0.4164032 , 0.5969664 , 0.47507906,
       0.5946119 , 0.4618724 , 0.499723  , 0.46639863, 0.5154557 ,
       0.55471206, 0.5992708 , 0.41802582, 0.43382123, 0.69340426,
       0.6037512 , 0.44342098, 0.5387671 , 0.62244385, 0.63985544,
       0.5230318 , 0.5624724 ], dtype=float32)
Coordinates:
  * season_year  (season_year) int32 128B 1990 1991 1992 1993 ... 2019 2020 2021

In [13]:
mjja_qrunoff/mjja_precip

<xarray.DataArray (season_year: 32)> Size: 128B
array([1.4851295, 1.0064741, 1.7061015, 1.1655351, 1.3959974, 1.3001654,
       1.1234043, 1.3640095, 1.1984773, 1.6754297, 1.8030093, 1.7304842,
       1.6234131, 1.2141713, 1.3801774, 1.1766162, 1.4696887, 1.3400465,
       1.5384433, 1.3672144, 1.287473 , 1.0519626, 1.7141573, 1.6995903,
       1.5118159, 1.164055 , 1.554053 , 1.1786325, 1.0377561, 1.2413017,
       1.432974 , 1.3052732], dtype=float32)
Coordinates:
  * season_year  (season_year) int32 128B 1990 1991 1992 1993 ... 2019 2020 2021